In [24]:
from langchain import hub
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyMuPDFLoader
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_community.llms import Together
from langchain_core.prompts import PromptTemplate
import os
from dotenv import load_dotenv

In [25]:
loader = PyMuPDFLoader("../DocumentStore/The_Art_Of_War.pdf")
pages = loader.load()

In [26]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, 
                                               chunk_overlap = 200, 
                                                length_function=len,
                                                is_separator_regex=False,
                                                separators=[
                                                            "\n\n",
                                                            "\n",
                                                            ".",
                                                            ",",
                                                            "\u200B",  # Zero-width space
                                                            "\uff0c",  # Fullwidth comma
                                                            "\u3001",  # Ideographic comma
                                                            "\uff0e",  # Fullwidth full stop
                                                            "\u3002",  # Ideographic full stop
                                                            " ",
                                                            "",
                                                        ]
                                            )
splits = text_splitter.split_documents(pages)

In [1]:
# splits[1]

In [28]:
embedding_function = SentenceTransformerEmbeddings(model_name="BAAI/bge-base-en-v1.5")

In [29]:
vectorstore = Chroma.from_documents(documents=splits, embedding=embedding_function, persist_directory="../chroma_db")

In [30]:
query = "Sun Tzu"
k = 10
results = vectorstore.similarity_search_with_relevance_scores(query, k=k)

In [31]:
k = 5
retriever = vectorstore.as_retriever(search_kwargs={"k":k})
query = "How update MSME UDYAM details?"
results = retriever.invoke(query)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [32]:
load_dotenv()

prompt = PromptTemplate(input_variables=['context', 'question'],
                        template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Keep answers descriptive and mention the process.\nQuestion: {question} \nContext: {context} \nAnswer:")

response = Together(
        model="mistralai/Mistral-7B-Instruct-v0.1",
        together_api_key= os.environ["TOGETHER_API_KEY"],
        temperature=0.3,
        max_tokens=512
    )

rag_chain = (
    {"context": retriever 
    | format_docs, "question": RunnablePassthrough()}
    | prompt
    | response
    | StrOutputParser())

In [33]:
query = "What is the functionality of Reconcilation statement:missing in books?"

query_list = ["Describe the ledger statement data-table?", ]
# query = "Summarise the document for me in 500 words"
ls = rag_chain.invoke(query)
print(ls)

 The functionality of Reconcilation statement:missing in books is not specified in the provided context.
